## Project 3 - Predicting Roadway Congestion

## Setting Up

In [1]:
import pandas as pd
import numpy as np

from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import sqlalchemy

import requests
import sys
import pickle

In [2]:
sqlalchemy.__version__

'1.3.5'

### Connecting to PostgreSQL in an AWS EC2 Instance

In [3]:
params = {
    'host': 'ec2-18-188-138-161.us-east-2.compute.amazonaws.com',
    'user': 'ubuntu',
    'port': 5432
}

In [9]:
connection = connect(**params, dbname='traffic_p3')
cursor = connection.cursor()

In [5]:
# cursor.execute("""SELECT * FROM data
#                LIMIT 10;""")
# cursor.fetchall()

In [6]:
# query = """
# SELECT data.report_id, timestamp, 
#         avgspeed, vehiclecount,
#         avgmeasuredtime, medianmeasuredtime,ndt_in_kmh,
#         avgspeed - ndt_in_kmh as "congestion"
# FROM data
# LEFT JOIN metadata on data.report_id = metadata.report_id
# """

# cursor.execute(query)
# cursor.fetchmany(10)

In [7]:
# connection.rollback()

### Pulling data from a database into a Pandas Dataframe

In [10]:
traf_init = pd.read_sql("""
SELECT data.report_id, timestamp 
        , avgspeed AS "Obs_Speed" 
        , ndt_in_kmh AS "Nom_Speed"
        , avgmeasuredtime AS "Av_Obs_Time"
        , medianmeasuredtime AS "Med_Obs_Time"
        , duration_in_sec AS "Nom_Time"
        , vehiclecount AS "Vehicles"
        , CAST(avgspeed AS FLOAT)/ndt_in_kmh AS "Con_Speed"
        , CAST(avgmeasuredtime AS FLOAT)/duration_in_sec AS "Con_AvTime"
        , CAST(medianmeasuredtime AS FLOAT)/duration_in_sec "Con_MedTime"
        , road_type
        , distance_in_meters AS "Distance"
FROM data
LEFT JOIN metadata on data.report_id = metadata.report_id
/*WHERE avgspeed <> 0 */
""", connection)

In [11]:
traf_init.head()

,report_id,timestamp,Obs_Speed,Nom_Speed,Av_Obs_Time,Med_Obs_Time,Nom_Time,Vehicles,Con_Speed,Con_AvTime,Con_MedTime,road_type,Distance
0,158324,2014-03-07T03:45:00,60,71,61,61,52,0,0.845070,1.173077,1.173077,MAJOR_ROAD,1030
1,158324,2014-03-07T03:50:00,60,71,61,61,52,0,0.845070,1.173077,1.173077,MAJOR_ROAD,1030
2,158324,2014-03-07T03:55:00,74,71,50,50,52,1,1.042254,0.961538,0.961538,MAJOR_ROAD,1030
3,158324,2014-03-07T04:00:00,74,71,50,50,52,1,1.042254,0.961538,0.961538,MAJOR_ROAD,1030
4,158324,2014-03-07T04:05:00,66,71,56,56,52,2,0.929577,1.076923,1.076923,MAJOR_ROAD,1030


In [12]:
traf_init.describe()

,report_id,Obs_Speed,Nom_Speed,Av_Obs_Time,Med_Obs_Time,Nom_Time,Vehicles,Con_Speed,Con_AvTime,Con_MedTime,Distance
count,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07,2.509754e+07
mean,1.904559e+05,4.363956e+01,4.470692e+01,1.070772e+02,1.070772e+02,9.153759e+01,3.155117e+00,9.902374e-01,1.187958e+00,1.187958e+00,1.150786e+03
std,1.169769e+04,2.192994e+01,1.624141e+01,1.764342e+02,1.764342e+02,5.871725e+01,5.864022e+00,4.460763e-01,1.642894e+00,1.642894e+00,1.071866e+03
min,1.583240e+05,0.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,3.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.210000e+02
25%,1.846210e+05,2.900000e+01,3.500000e+01,5.300000e+01,5.300000e+01,5.900000e+01,0.000000e+00,7.551020e-01,7.741935e-01,7.741935e-01,6.370000e+02
50%,1.907700e+05,4.200000e+01,4.100000e+01,8.000000e+01,8.000000e+01,8.000000e+01,1.000000e+00,1.000000e+00,9.619048e-01,9.619048e-01,8.900000e+02
75%,1.979770e+05,5.700000e+01,4.900000e+01,1.160000e+02,1.160000e+02,1.060000e+02,4.000000e+00,1.230769e+00,1.238636e+00,1.238636e+00,1.258000e+03
max,2.101990e+05,1.500000e+02,1.170000e+02,3.656000e+03,3.656000e+03,7.160000e+02,1.210000e+02,7.842105e+00,4.970690e+01,4.970690e+01,1.306800e+04


In [13]:
traf_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25097541 entries, 0 to 25097540
Data columns (total 13 columns):
report_id       int64
timestamp       object
Obs_Speed       int64
Nom_Speed       int64
Av_Obs_Time     int64
Med_Obs_Time    int64
Nom_Time        int64
Vehicles        int64
Con_Speed       float64
Con_AvTime      float64
Con_MedTime     float64
road_type       object
Distance        int64
dtypes: float64(3), int64(8), object(2)
memory usage: 2.4+ GB


## Pickling the data for next steps

In [14]:
with open('traf_init.pickle', 'wb') as to_write:
    pickle.dump(traf_init, to_write)

## Misc (Used Elsewhere, Commented Out)

In [15]:
# traf_sample = traf_full.sample(n = 7000, random_state = 13 )

In [16]:
# traf_sample

In [17]:
# traf_sample.describe()

In [18]:
# traf_sample.info()

In [19]:
# with open('traf_sample.pickle', 'wb') as to_write:
#     pickle.dump(traf_sample, to_write)